<a href="https://colab.research.google.com/github/njw8jx/ds3001group/blob/main/north%20carolina/NC%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Read in files

outcomes = pd.read_excel('WinningPartyByCounties.xlsx') # party outcome at county level

rawdemo2012 = pd.read_excel('demographicsallstates2012.xlsx') # demographic variables at county level 2012
rawdemo2016 = pd.read_excel('demographicsallstates2016.xlsx') # demographic variables at county level 2016
rawdemo2020 = pd.read_excel('demographicsallstates2020u.xlsx') # demographic variables at county level 2020

ncpoverty = pd.read_csv('ncpoverty.csv') # poverty variable at county level -- Nevada
ncpoverty = ncpoverty.drop([0,1, 38, 39]) # Drop national and state level statistics

prediction = pd.read_excel('2022data.xlsx')
prediction.head(10)

,GIS Join Match Code,Data File Year,State/US-Abbreviation (USPS),Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,Aggregate income in the past 12 months (in 2022 inflation-adjusted dollars),Total.5,In labor force,In labor force Percentage,In labor force: Civilian labor force,In labor force: Civilian labor force: Employed,In labor force: Civilian labor force: Unemployed,In labor force: Armed Forces,Not in labor force,Not in labor force Percentage
0,G0400010,2018-2022,AZ,NaN,NaN,Arizona,4.0,Apache County,1.0,NaN,...,2076142800,47146,28106,0.596148,27163,26411,752,943,19040,0.403852
1,G0400030,2018-2022,AZ,NaN,NaN,Arizona,4.0,Cochise County,3.0,NaN,...,9081671900,189569,111766,0.589580,111079,107254,3825,687,77803,0.410420
2,G0400050,2018-2022,AZ,NaN,NaN,Arizona,4.0,Coconino County,5.0,NaN,...,581565400,20399,8968,0.439629,8968,8452,516,0,11431,0.560371
3,G0400070,2018-2022,AZ,NaN,NaN,Arizona,4.0,Gila County,7.0,NaN,...,555512400,18046,9028,0.500277,9028,8242,786,0,9018,0.499723
4,G0400090,2018-2022,AZ,NaN,NaN,Arizona,4.0,Graham County,9.0,NaN,...,1716337800,47261,26459,0.559849,26434,24856,1578,25,20802,0.440151
5,G0400110,2018-2022,AZ,NaN,NaN,Arizona,4.0,Greenlee County,11.0,NaN,...,228401300,8376,3888,0.464183,3888,3744,144,0,4488,0.535817
6,G0400120,2018-2022,AZ,NaN,NaN,Arizona,4.0,La Paz County,12.0,NaN,...,499854700,15203,8047,0.529303,8043,7495,548,4,7156,0.470697
7,G0400130,2018-2022,AZ,NaN,NaN,Arizona,4.0,Maricopa County,13.0,NaN,...,3278894900,94627,53521,0.565600,53147,49455,3692,374,41106,0.434400
8,G0400150,2018-2022,AZ,NaN,NaN,Arizona,4.0,Mohave County,15.0,NaN,...,934715300,28036,15691,0.559673,15631,15072,559,60,12345,0.440327
9,G0400170,2018-2022,AZ,NaN,NaN,Arizona,4.0,Navajo County,17.0,NaN,...,712518400,20956,10713,0.511214,10713,10288,425,0,10243,0.488786


In [7]:
# Filter files for NC

ncoutcomes = outcomes[outcomes['state'] == 'NORTH CAROLINA'] # filtered outcome variable for North Carolina

demo2012 = rawdemo2012[rawdemo2012['State Name'] == 'North Carolina']
demo2016 = rawdemo2016[rawdemo2016['State Name'] == 'North Carolina']
demo2020 = rawdemo2020[rawdemo2020['State Name'] == 'North Carolina'] # filtered demographic variables for North Carolina

In [10]:
# Separate outcomes & poverty by year

ncoutcomes2012 = ncoutcomes[ncoutcomes['year'] == 2012]
ncoutcomes2016 = ncoutcomes[ncoutcomes['year'] == 2016]
ncoutcomes2020 = ncoutcomes[ncoutcomes['year'] == 2020] # separated outcomes variable by year

ncpoverty2012 = ncpoverty[ncpoverty['Year'] == 2012]
ncpoverty2016 = ncpoverty[ncpoverty['Year'] == 2016]
ncpoverty2020 = ncpoverty[ncpoverty['Year'] == 2020] # separated poverty variable by year

In [11]:
# Merge

election2012 = pd.merge(demo2012, ncpoverty2012, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2012 = pd.merge(election2012, ncoutcomes2012, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2012
# now have a complete file for 2012

election2016 = pd.merge(demo2016, ncpoverty2016, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2016 = pd.merge(election2016, ncoutcomes2016, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2016
# now have a complete file for 2016

election2020 = pd.merge(demo2020, ncpoverty2020, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2020 = pd.merge(election2020, ncoutcomes2020, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2012
# now have a complete file for 2020


In [19]:
# Prepare prediction data

ncprediction = prediction[prediction['State Name'] == 'North Carolina']

ncprediction = ncprediction[['County Name','Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage',]]

ncprediction.head(100)
# Prediction data ready. Missing poverty statistics and median income statistics

,County Name,Males Under 18 Percentage,Males 18-24 Percentage,Males 25-34 Percentage,Males 35-44 Percentage,Males 45-64 Percentage,Males 65+ Percentage,Females Under 18 Percentage,Females 18-24 Percentage,Females 25-34 Percentage,...,Bachelor's degree Percentage,Master's degree Percentage,Professional school degree Percentage,Doctorate degree Percentage,"less than $39,999 Percentage","$40,000-$74,999 Percentage","$75,000-$99,999 Percentage","$100,000-$149,999 Percentage","$150,000-$199,999 Percentage","more than $200,000 Percentage"
274,Alamance County,0.112505,0.050588,0.061113,0.056724,0.125993,0.072343,0.628307,0.055536,0.064851,...,0.189833,0.061861,0.009340,0.012312,0.531783,0.193798,0.091473,0.119380,0.024806,0.038760
275,Alexander County,0.102178,0.046295,0.063635,0.065772,0.146692,0.093056,0.576880,0.034762,0.052760,...,0.110369,0.035349,0.003755,0.003755,0.159650,0.197687,0.128090,0.211808,0.130552,0.172214
276,Alleghany County,0.074529,0.031577,0.056966,0.047320,0.144690,0.130858,0.602603,0.031122,0.050414,...,0.134344,0.060766,0.004937,0.007405,0.381125,0.313975,0.156080,0.054446,0.058076,0.036298
277,Anson County,0.097838,0.051441,0.072523,0.067297,0.133108,0.084369,0.595495,0.037387,0.058874,...,0.087397,0.022293,0.001963,0.002217,0.308157,0.319570,0.110775,0.153072,0.054716,0.053709
278,Ashe County,0.088045,0.033372,0.051235,0.056766,0.144886,0.121006,0.585672,0.030046,0.052132,...,0.126004,0.064560,0.005891,0.011442,0.200564,0.286218,0.137822,0.206909,0.103631,0.064857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,Wayne County,0.119816,0.055039,0.070880,0.058010,0.122523,0.071672,0.616718,0.044263,0.064573,...,0.138754,0.048794,0.006746,0.005609,0.260490,0.253414,0.133872,0.173070,0.077836,0.101317
370,Wilkes County,0.102715,0.040076,0.055894,0.056287,0.142745,0.100038,0.599743,0.037535,0.055108,...,0.116221,0.035767,0.004670,0.007183,0.291737,0.284058,0.150788,0.157382,0.062796,0.053240
371,Wilson County,0.116033,0.042572,0.057368,0.055563,0.123673,0.082449,0.634154,0.042025,0.062428,...,0.130399,0.058756,0.009022,0.006471,0.261441,0.232758,0.124888,0.167135,0.080440,0.133338
372,Yadkin County,0.111159,0.045091,0.060381,0.050644,0.144179,0.090397,0.593294,0.037098,0.056384,...,0.107107,0.026089,0.004486,0.003468,0.309112,0.265636,0.128840,0.152983,0.075063,0.068366


In [20]:
# Put together data across years

elections = pd.concat([election2012, election2016, election2020], axis=0, ignore_index=True)
finalelections = elections[['party', 'Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage']]

In [21]:
# Train-test split
from sklearn.model_selection import train_test_split

# Process data:
X = finalelections[['Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage']]
y = finalelections['party']

# Train-test split:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=100)
print(f"X_train shape: {X_train.shape}") # got it done

X_train shape: (244, 51)


In [22]:
# Finding missing data

total_nan_count = finalelections.isna().sum().sum()
print("Total NaNs in the DataFrame:")
print(total_nan_count)

columns_with_nan = finalelections.loc[:, finalelections.isna().any(axis=0)]
print("Columns with NaNs:")
print(columns_with_nan)

Total NaNs in the DataFrame:
412
Columns with NaNs:
          party  Males Under 18 Percentage  Males 18-24 Percentage  \
0           NaN                        NaN                     NaN   
1           NaN                        NaN                     NaN   
2    REPUBLICAN                   0.250834                0.104665   
3    REPUBLICAN                   0.227725                0.079354   
4    REPUBLICAN                   0.196114                0.073947   
..          ...                        ...                     ...   
301    DEMOCRAT                   0.243682                0.093594   
302  REPUBLICAN                   0.223439                0.081065   
303  REPUBLICAN                   0.206463                0.072109   
304         NaN                   0.196913                0.091007   
305         NaN                   0.191856                0.066700   

     Males 25-34 Percentage  Males 35-44 Percentage  Males 45-64 Percentage  \
0                       NaN 

In [23]:
# Missing data problem

finalelections = finalelections.drop([34,35])
finalelections = finalelections.fillna(0)

In [24]:
# Recount missing data

total_nan_count2 = finalelections.isna().sum().sum()
print("Total NaNs in the DataFrame:")
print(total_nan_count2)

columns_with_nan2 = finalelections.loc[:, finalelections.isna().any(axis=0)]
print("Columns with NaNs:")
print(columns_with_nan2)

Total NaNs in the DataFrame:
0
Columns with NaNs:
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, ...]

[304 rows x 0 columns]


In [26]:
# Find NaNs in X_train
nan_count_X_train = X_train.isna().sum().sum()
print(f"Number of NaNs in X_train: {nan_count_X_train}")

# Find NaNs in y_train
nan_count_y_train = y_train.isna().sum()
print(f"Number of NaNs in y_train: {nan_count_y_train}")

Number of NaNs in X_train: 329
Number of NaNs in y_train: 5


In [27]:
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)

In [28]:
# Find NaNs in X_train
nan_count_X_train = X_train.isna().sum().sum()
print(f"Number of NaNs in X_train: {nan_count_X_train}")

# Find NaNs in y_train
nan_count_y_train = y_train.isna().sum()
print(f"Number of NaNs in y_train: {nan_count_y_train}")

Number of NaNs in X_train: 0
Number of NaNs in y_train: 0


In [44]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

le = LabelEncoder()

# Convert y_train and y_test to string type before fitting LabelEncoder
y_train = y_train.astype(str)
y_test = y_test.astype(str)

# Replace NaN values with a placeholder string (e.g., 'unknown') in both y_train and y_test
# using np.where to handle NaN values in NumPy arrays
y_train = np.where(pd.isnull(y_train), 'unknown', y_train) # using np.where to apply conditional logic and fillna
y_test = np.where(pd.isnull(y_test), 'unknown', y_test)  # using np.where to apply conditional logic and fillna

# Fit and transform the target variable (y_train and y_test)
y_train = le.fit_transform(y_train)

# Ensure all labels in y_test are in the classes_ of the LabelEncoder
# Get unique labels in y_test
unique_y_test = set(y_test)

# Get labels seen during training
trained_labels = set(le.classes_)

# Find unseen labels
unseen_labels = unique_y_test - trained_labels

# If there are unseen labels, add them to the LabelEncoder's classes_
if unseen_labels:
    # Extend classes_ with unseen labels
    le.classes_ = np.append(le.classes_, list(unseen_labels))


y_test = le.transform(y_test)  # Transform y_test using the same encoder

# Fit model:
model = RandomForestClassifier()  # Build a random forest model
rf = model.fit(X_train, y_train)  # Fit the rf model

In [45]:
# Select the relevant columns from azprediction that match the training data
ncprediction_subset = ncprediction[['Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage',]]

# Use the trained model to make predictions on the selected data
predictions = rf.predict(ncprediction_subset)

# Inverse transform the predictions to get the original party labels
predicted_parties = le.inverse_transform(predictions)

# Add the predictions to the prediction DataFrame
ncprediction['predicted_party'] = predicted_parties

# Print or analyze the results
print(ncprediction[['County Name', 'predicted_party']])

          County Name predicted_party
274   Alamance County               2
275  Alexander County               2
276  Alleghany County               2
277      Anson County               1
278       Ashe County               2
..                ...             ...
369      Wayne County               2
370     Wilkes County               2
371     Wilson County               1
372     Yadkin County               2
373     Yancey County               2

[100 rows x 2 columns]


In [47]:
# Replace numerical party representation with string labels
ncprediction['predicted_party'] = ncprediction['predicted_party'].replace({'1': 'DEMOCRAT', '2': 'REPUBLICAN'})

# Print or analyze the results
print(ncprediction[['County Name', 'predicted_party']])

          County Name predicted_party
274   Alamance County      REPUBLICAN
275  Alexander County      REPUBLICAN
276  Alleghany County      REPUBLICAN
277      Anson County        DEMOCRAT
278       Ashe County      REPUBLICAN
..                ...             ...
369      Wayne County      REPUBLICAN
370     Wilkes County      REPUBLICAN
371     Wilson County        DEMOCRAT
372     Yadkin County      REPUBLICAN
373     Yancey County      REPUBLICAN

[100 rows x 2 columns]


In [48]:
print(ncoutcomes2020[['county_name', 'party']])

     county_name       party
1300    ALAMANCE  REPUBLICAN
1301   ALEXANDER  REPUBLICAN
1302   ALLEGHANY  REPUBLICAN
1303       ANSON    DEMOCRAT
1304        ASHE  REPUBLICAN
...          ...         ...
1395       WAYNE  REPUBLICAN
1396      WILKES  REPUBLICAN
1397      WILSON    DEMOCRAT
1398      YADKIN  REPUBLICAN
1399      YANCEY  REPUBLICAN

[100 rows x 2 columns]
